In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [55]:
uchicago = pd.read_csv("season_plays23.csv").iloc[:, 1:]
w_uchicago = pd.read_csv("w_season_plays23.csv").iloc[:, 1:]

In [56]:
def points_scored(action):
    if "3-pt" in action:
        return 3
    elif "free throw" in action:
        return 1
    else:
        return 2
    
def calculate_scores(df, lineup):
    current = df[df.Lineup == lineup]
    
    makes = current[current.Action.str.contains('made')]
    makes["Points"] = makes.Action.apply(points_scored)

    chicago_scores = makes[makes.Chicago == makes.Team].Points.sum()
    other_scores = makes[makes.Chicago != makes.Team].Points.sum()

    return (chicago_scores, other_scores)

def calculate_play(df, lineup, play):
    current = df[df.Lineup == lineup]
    plays = current[(current.Action.str.contains(play))]

    chicago = plays[(plays.Chicago == plays.Team)].shape[0]
    other = plays[(plays.Chicago != plays.Team)].shape[0]
    
    return (chicago, other)

def calculate_makes(df, lineup, three=False):
    current = df[df.Lineup == lineup]
    
    makes = current[(current.Action.str.contains("made")) & ~(current.Action.str.contains("free throw"))]
    if three:
        makes = makes[makes.Action.str.contains("3-pt")]
    
    chicago_makes = makes[(makes.Chicago == makes.Team)].shape[0]
    other_makes = makes[(makes.Chicago != makes.Team)].shape[0]
    
    return (chicago_makes, other_makes)

def calculate_percentage(df, lineup, three=False, ft=False, att=False):
    current = df[df.Lineup == lineup]
    misses = current[(current.Action.str.contains("missed"))]
    makes = current[(current.Action.str.contains("made"))]
    
    if three:
        misses = misses[misses.Action.str.contains("3-pt")]
        makes = makes[makes.Action.str.contains("3-pt")]
    elif ft:
        misses = misses[misses.Action.str.contains("free throw")]
        makes = makes[makes.Action.str.contains("free throw")]
    else:
        misses = misses[~(misses.Action.str.contains("free throw"))]
        makes = makes[~(makes.Action.str.contains("free throw"))]
        
    chicago_misses = misses[(misses.Chicago == misses.Team)].shape[0]
    chicago_makes = makes[(makes.Chicago == makes.Team)].shape[0]
    other_misses = misses[(misses.Chicago != misses.Team)].shape[0]
    other_makes = makes[(makes.Chicago != makes.Team)].shape[0]
    
    chicago_attempts = chicago_misses + chicago_makes
    other_attempts = other_misses + other_makes
    
    if att:
        return (chicago_attempts, other_attempts)
    
    if (chicago_attempts == 0) and (other_attempts == 0):
        return (0, 0)
    elif chicago_attempts == 0:
        return (0, other_makes / (other_misses + other_makes))
    elif other_attempts == 0:
        return (chicago_makes / (chicago_misses + chicago_makes), 0)
    else:
        return (chicago_makes / (chicago_misses + chicago_makes), 
                other_makes / (other_misses + other_makes))
    
def sec_to_time(seconds):
    minutes = str(seconds // 60)
    
    if (seconds % 60) < 10:
        return minutes + ":0" + str(seconds % 60)
    else:
        return minutes + ":" + str(seconds % 60)

def time_passed(start, end):
    if start == '30300':
        start = "00:30"
    elif end == '30300':
        end = "00:30"
    
    start_time = int(start.split(":")[0])*60 + int(start.split(":")[1])
    end_time = int(end.split(":")[0])*60 + int(end.split(":")[1])
    
    if start_time > end_time:
        return start_time - end_time
    else:
        return 0

def calculate_time_played(df, lineup):
    total_time = 0
    for x in range(len(df[df.Lineup == lineup])-1):
        current_play = df[df.Lineup == lineup].reset_index(drop=False).values[x]
        next_play = df[df.Lineup == lineup].reset_index(drop=False).values[x+1]
        if x > 0:
            last_play = df[df.Lineup == lineup].reset_index(drop=False).values[x-1]
        else:
            last_play = current_play

        if (current_play[0] != next_play[0] - 1) or (current_play[0] == 342):
            continue

        if (current_play[0] == 0) or (last_play[4] != current_play[4]):
            if (current_play[4] == 3):
                start = "05:00"
            else:
                start = team_start
            end = next_play[5]
        elif next_play[4] != current_play[4]:
            start = current_play[5]
            end = "00:00"
        else:
            start = current_play[5]
            end = next_play[5]

        total_time += time_passed(start, end)

    return total_time / 60

def analyze_lineup(df, lineup):
    return [calculate_time_played(df, lineup), # MP
            
            calculate_scores(df, lineup)[0], # PTS
            calculate_scores(df, lineup)[1], # OPP PTS
            
            calculate_scores(df, lineup)[0] - calculate_scores(df, lineup)[1], # +/-
            
            calculate_play(df, lineup, "Assist")[0], # AST
            
            calculate_play(df, lineup, "offensive rebound")[0], # OREB
            calculate_play(df, lineup, "offensive rebound")[1], # OPP OREB
            calculate_play(df, lineup, "defensive rebound")[0], # DREB
            calculate_play(df, lineup, "defensive rebound")[1], # OPP DREB
            
            calculate_makes(df, lineup)[0], # FGM
            calculate_percentage(df, lineup, att=True)[0], # FGA
            calculate_makes(df, lineup)[1], # OPP FGM
            calculate_percentage(df, lineup, att=True)[1], # OPP FGA
            
            calculate_makes(df, lineup, three=True)[0], # 3PM
            calculate_percentage(df, lineup, three=True, att=True)[0], # 3PA
            calculate_makes(df, lineup, three=True)[1], # OPP 3PM
            calculate_percentage(df, lineup, three=True, att=True)[1], # OPP 3PA
            
            calculate_percentage(df, lineup, ft=True, att=True)[0], # FTA
            calculate_percentage(df, lineup, ft=True, att=True)[1], # OPP FTA
            
            calculate_play(df, lineup, "Turnover")[0], # TO
            calculate_play(df, lineup, "Turnover")[1],# OPP TO
            
            calculate_play(df, lineup, "Steal")[0], # STL
            calculate_play(df, lineup, "Block")[0], # BLK
            
            calculate_play(df, lineup, "Foul")[0], # PF
            calculate_play(df, lineup, "Foul")[1]] # OPP PF

def pbp_lineups(game_pbp):
    date = game_pbp.Date.values[0]
    lineups = [l for l in pd.unique(game_pbp.Lineup) if len(l.split(", ")) == 5]

    lineup_data = []
    for l in lineups:
        lineup_data.append( [l] + analyze_lineup(game_pbp, l) )

    lineup_analysis = pd.DataFrame(lineup_data, 
                                   columns = ['Lineup',
                                              'MP', 
                                              'PTS', 'OPP PTS', 
                                              '+/-', 
                                              'AST',
                                              'OREB', 'OPP OREB', 'DREB', 'OPP DREB',
                                              'FGM', 'FGA', 'OPP FGM', 'OPP FGA',
                                              '3PM', '3PA', 'OPP 3PM', 'OPP 3PA',
                                              'FTA', 'OPP FTA', 
                                              'TO', 'OPP TO', 
                                              'STL', 'BLK',
                                              'PF', 'OPP PF'])
    
    lineup_analysis['Lineup_Set'] = lineup_analysis.Lineup.apply(lambda i : set(i.split(", ")))
    lineup_analysis['Lineup'] = lineup_analysis['Lineup_Set'].apply(lambda i : ", ".join(sorted(i)))
    lineup_analysis = lineup_analysis.groupby('Lineup').sum().reset_index(drop=False)
    
    return lineup_analysis

In [57]:
def add_advanced(lineup_analysis):
    lineup_analysis['FG%'] = 100 * lineup_analysis['FGM'] / lineup_analysis['FGA']
    lineup_analysis['OPP FG%'] = 100 * lineup_analysis['FGM'] / lineup_analysis['FGA']
    lineup_analysis['3P%'] = 100 * lineup_analysis['3PM'] / lineup_analysis['3PA']
    lineup_analysis['OPP 3P%'] = 100 * lineup_analysis['OPP 3PM'] / lineup_analysis['OPP 3PA']
    
    # formula from nbastuffer.com
    lineup_analysis['POSS'] = 0.96 * (lineup_analysis.FGA +
                                      lineup_analysis.TO + 
                                      0.44*(lineup_analysis.FTA) - 
                                      lineup_analysis.OREB)
    lineup_analysis['OPP POSS'] = 0.96 * (lineup_analysis['OPP FGA'] +
                                          lineup_analysis['OPP TO'] + 
                                          0.44*(lineup_analysis['OPP FTA']) - 
                                          lineup_analysis['OPP OREB'])

    lineup_analysis['PPP'] = lineup_analysis.PTS / lineup_analysis.POSS
    lineup_analysis['OPP PPP'] = lineup_analysis['OPP PTS'] / lineup_analysis['OPP POSS']

    lineup_analysis['PTS DIFF PER40'] = 40 * (lineup_analysis['PTS'] - lineup_analysis['OPP PTS']) / lineup_analysis['MP']

    lineup_analysis['REB'] = lineup_analysis.OREB + lineup_analysis.DREB
    lineup_analysis['OPP REB'] = lineup_analysis['OPP OREB'] + lineup_analysis['OPP DREB']
    lineup_analysis['ORB%'] = 100 * lineup_analysis.OREB / (lineup_analysis.OREB + lineup_analysis['OPP DREB'])
    lineup_analysis['DRB%'] = 100 * lineup_analysis.DREB / (lineup_analysis.DREB + lineup_analysis['OPP OREB'])
    lineup_analysis['RB%'] = 100 * lineup_analysis.REB / (lineup_analysis.REB + lineup_analysis['OPP REB'])
    lineup_analysis['OPP ORB%'] = 100 * lineup_analysis['OPP OREB'] / (lineup_analysis['OPP OREB'] + lineup_analysis.DREB)
    lineup_analysis['OPP DRB%'] = 100 * lineup_analysis['OPP DREB'] / (lineup_analysis['OPP DREB'] + lineup_analysis.OREB)
    lineup_analysis['OPP RB%'] = 100 * lineup_analysis['OPP REB'] / (lineup_analysis['OPP REB'] + lineup_analysis.REB)

    lineup_analysis['TO%'] = 100 * lineup_analysis.TO / (lineup_analysis.FGA + 
                                                         0.44*(lineup_analysis.FTA) + 
                                                         lineup_analysis.TO)
    lineup_analysis['OPP TO%'] = 100 * lineup_analysis['OPP TO'] / (lineup_analysis['OPP FGA'] + 
                                                         0.44*(lineup_analysis['OPP FTA']) + 
                                                         lineup_analysis['OPP TO'])
    
    lineup_analysis['POSS PER40'] = 40 * lineup_analysis['POSS'] / lineup_analysis['MP']
    
    lineup_analysis = lineup_analysis.round(2)
    lineup_analysis['POSS'] = lineup_analysis['POSS'].round(1)
    lineup_analysis['OPP POSS'] = lineup_analysis['OPP POSS'].round(1)
    lineup_analysis['ORB%'] = lineup_analysis['ORB%'].round(1)
    lineup_analysis['DRB%'] = lineup_analysis['DRB%'].round(1)
    lineup_analysis['RB%'] = lineup_analysis['RB%'].round(1)
    lineup_analysis['TO%'] = lineup_analysis['TO%'].round(1)
    lineup_analysis['OPP ORB%'] = lineup_analysis['OPP ORB%'].round(1)
    lineup_analysis['OPP DRB%'] = lineup_analysis['OPP DRB%'].round(1)
    lineup_analysis['OPP RB%'] = lineup_analysis['OPP RB%'].round(1)
    lineup_analysis['OPP TO%'] = lineup_analysis['OPP TO%'].round(1)
    
    return lineup_analysis

def pbp_helper(df):
    lineup_data = []
    for x in pd.unique(df.Opponent):
        lineup_data.append( pbp_lineups( df[(df.Opponent == x)] ) )
        
    lineup_analysis = pd.concat(lineup_data)
    lineup_analysis = lineup_analysis.groupby('Lineup').sum()
        
    lineup_analysis = add_advanced(lineup_analysis)
    
    lineup_analysis = lineup_analysis.fillna(0).sort_values(by='MP', ascending=False)
    
    lineup_analysis['Lineup'] = lineup_analysis.index
    lineup_analysis.Lineup = lineup_analysis.Lineup.apply(lambda l : ", ".join([i.split(" ")[0] for i in l.split(", ")]))
    
    return lineup_analysis.set_index('Lineup')

def game_helper( game_pbp ):
    df = game_pbp.copy()
    df['Lineup'] = df.Opponent.apply(lambda i : ", ".join([i] * 5))

    lineup_data = []
    for x in pd.unique(df.Game):
        lineup_data.append( [pbp_lineups( df[(df.Game == x)] ), x] )

    lineup_analysis = pd.concat( [i[0] for i in lineup_data] )
    lineup_analysis['Game'] = [i[1] for i in lineup_data]
    lineup_analysis = lineup_analysis.groupby('Lineup').sum()

    lineup_analysis = add_advanced(lineup_analysis)
    
    lineup_analysis = lineup_analysis.sort_values(by='Game')
    lineup_analysis = lineup_analysis[['POSS', 'OPP POSS', 'PPP', 'OPP PPP', 'PTS DIFF PER40', 'REB', 'OPP REB', 'ORB%', 'DRB%', 'RB%', 'TO%', 'OPP TO%']]
    
    return lineup_analysis.fillna(0)

def total_advanced(game_pbp):
    df = game_pbp.copy()
    df['Lineup'] = ", ".join(['Total']*5)
    
    lineup_analysis = pbp_lineups(df)
    
    lineup_analysis = add_advanced(lineup_analysis)
    
    lineup_analysis['POSS'] = round(40 * lineup_analysis['POSS'] / lineup_analysis['MP'], 1)
    lineup_analysis['OPP POSS'] = round(40 * lineup_analysis['OPP POSS'] / lineup_analysis['MP'], 1)
    
    lineup_analysis = lineup_analysis[['POSS', 'OPP POSS', 'PPP', 'OPP PPP', 'PTS DIFF PER40', 'REB', 'OPP REB', 'ORB%', 'DRB%', 'RB%', 'TO%', 'OPP TO%']]
    lineup_analysis.index = ['Total']
    
    return lineup_analysis

In [58]:
team_start = "20:00"
current = uchicago
opponent = "Ill. Wesleyan"

lineups = pbp_helper( current )
game_advanced = pd.concat([game_helper( current ), total_advanced( current )])

In [59]:
pbp_helper( current[(current.Opponent == opponent)] ).MP.sum()

44.04

In [60]:
lineups

,MP,PTS,OPP PTS,+/-,AST,OREB,OPP OREB,DREB,OPP DREB,FGM,FGA,OPP FGM,OPP FGA,3PM,3PA,OPP 3PM,OPP 3PA,FTA,OPP FTA,TO,OPP TO,STL,BLK,PF,OPP PF,FG%,OPP FG%,3P%,OPP 3P%,POSS,OPP POSS,PPP,OPP PPP,PTS DIFF PER40,REB,OPP REB,ORB%,DRB%,RB%,OPP ORB%,OPP DRB%,OPP RB%,TO%,OPP TO%,POSS PER40
Lineup,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Alec, Arrish, Elliot, Ezra, Thomas",35.98,66,40,26,12,11,7,31,23,27,60,16,51,9,25,5,21,4,7,11,10,4,2,9,10,45.00,45.00,36.00,23.81,59.3,54.8,1.11,0.73,28.90,42,30,32.4,81.6,58.3,18.4,67.6,41.7,15.1,15.6,65.91
"Alec, Arrish, Ezra, Joe, Thomas",24.00,34,27,7,7,8,10,20,17,13,36,13,42,4,16,1,12,5,1,9,4,2,3,4,9,36.11,36.11,25.00,8.33,37.6,35.0,0.90,0.77,11.67,28,27,32.0,66.7,50.9,33.3,68.0,49.1,19.1,8.6,62.72
"Bryce, Elliot, Ezra, Jackson, Thomas",16.00,20,24,-4,4,2,0,12,12,6,21,9,20,3,12,3,6,6,5,8,7,3,2,10,4,28.57,28.57,25.00,50.00,28.4,28.0,0.70,0.86,-10.00,14,12,14.3,100.0,53.8,0.0,85.7,46.2,25.3,24.0,71.14
"Alec, Arrish, Eamonn, Elliot, Joe",12.43,17,28,-11,3,3,2,7,5,7,15,11,20,2,6,2,5,1,4,9,3,2,1,5,1,46.67,46.67,33.33,40.00,20.6,21.8,0.83,1.28,-35.39,10,7,37.5,77.8,58.8,22.2,62.5,41.2,36.8,12.1,66.22
"Bryce, Elliot, Ezra, Josh, Thomas",10.10,16,14,2,5,3,1,6,7,7,17,7,14,1,5,0,2,1,0,3,5,4,1,1,6,41.18,41.18,20.00,0.00,16.7,17.3,0.96,0.81,7.92,9,8,30.0,85.7,52.9,14.3,70.0,47.1,14.7,26.3,66.31
"Eamonn, Joe, Josh, Leonardo, Skyler",9.78,22,14,8,6,3,2,6,3,10,16,4,12,2,3,2,5,0,4,2,4,2,1,3,0,62.50,62.50,66.67,40.00,14.4,15.1,1.53,0.93,32.71,9,5,50.0,75.0,64.3,25.0,50.0,35.7,11.1,22.5,58.88
"Alec, Ezra, Joe, Josh, Thomas",9.20,10,21,-11,3,4,2,4,7,4,14,8,14,1,6,4,8,2,1,2,1,1,0,3,3,28.57,28.57,16.67,50.00,12.4,12.9,0.81,1.63,-47.83,8,9,36.4,66.7,47.1,33.3,63.6,52.9,11.8,6.5,53.76
"Alec, Arrish, Eamonn, Joe, Thomas",8.73,21,18,3,4,3,6,2,4,7,12,5,13,2,4,0,3,7,9,2,5,2,1,5,7,58.33,58.33,50.00,0.00,13.5,15.3,1.55,1.17,13.74,5,10,42.9,25.0,33.3,75.0,57.1,66.7,11.7,22.8,61.91
"Bryce, Elliot, Jackson, Joe, Thomas",8.38,14,13,1,0,1,2,7,7,5,12,5,12,0,2,2,5,4,3,0,2,1,1,3,4,41.67,41.67,0.00,40.00,12.2,12.8,1.14,1.02,4.77,8,9,12.5,77.8,47.1,22.2,87.5,52.9,0.0,13.0,58.45


In [61]:
game_advanced

,POSS,OPP POSS,PPP,OPP PPP,PTS DIFF PER40,REB,OPP REB,ORB%,DRB%,RB%,TO%,OPP TO%
Lake Forest,61.6,63.9,1.27,1.05,11.05,40,29,29.6,76.2,58.0,16.6,7.8
Albion,58.5,59.4,0.85,0.71,8.05,44,41,39.5,64.3,51.8,20.5,22.1
Mt. St. Joseph,57.8,55.5,1.11,1.06,5.03,39,30,40.5,75.0,56.5,12.0,15.2
Colorado College,66.5,65.0,1.01,1.06,-2.01,35,39,24.1,62.2,47.3,26.2,11.8
Colby,64.7,66.0,0.94,1.11,-12.22,38,30,29.0,90.0,55.9,19.1,16.7
Wheaton (Ill.),57.8,58.4,0.95,1.25,-18.17,38,32,34.2,78.1,54.3,16.4,7.4
Kalamazoo,59.4,60.5,1.21,1.04,9.06,47,25,50.0,80.6,65.3,20.0,18.6
Edgewood,67.8,68.7,1.25,1.05,13.13,32,34,26.3,78.6,48.5,7.4,20.6
Illinois Tech,59.0,63.7,1.05,0.93,3.02,40,31,23.3,80.5,56.3,16.1,12.1
Ill. Wesleyan,68.1,64.9,0.93,1.03,-3.58,38,45,23.3,70.0,45.8,13.6,15.1


In [62]:
games = []
for opp in pd.unique(current.Opponent):
    opp_game = pbp_helper( current[(current.Opponent == opp)] )
    opp_game['Opponent'] = opp
    games.append(opp_game)
    
games = pd.concat(games)

games['Lineup'] = games.index
games.Lineup = games.Lineup.apply(lambda l : ", ".join([i.split(" ")[0] for i in l.split(", ")]))

games.set_index('Lineup')

,MP,PTS,OPP PTS,+/-,AST,OREB,OPP OREB,DREB,OPP DREB,FGM,FGA,OPP FGM,OPP FGA,3PM,3PA,OPP 3PM,OPP 3PA,FTA,OPP FTA,TO,OPP TO,STL,BLK,PF,OPP PF,FG%,OPP FG%,3P%,OPP 3P%,POSS,OPP POSS,PPP,OPP PPP,PTS DIFF PER40,REB,OPP REB,ORB%,DRB%,RB%,OPP ORB%,OPP DRB%,OPP RB%,TO%,OPP TO%,POSS PER40,Opponent
Lineup,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Alec, Arrish, Elliot, Ezra, Thomas",5.05,7,5,2,3,2,0,5,3,3,8,2,7,1,3,1,3,0,1,2,1,0,0,1,2,37.50,37.50,33.33,33.33,7.7,8.1,0.91,0.62,15.84,7,3,40.0,100.0,70.0,0.0,60.0,30.0,20.0,11.8,60.83,Lake Forest
"Alec, Arrish, Ezra, Skyler, Thomas",5.02,8,16,-8,1,1,2,3,2,3,6,6,11,1,2,4,8,2,0,1,0,0,0,1,2,50.00,50.00,50.00,50.00,6.6,8.6,1.21,1.85,-63.79,4,4,33.3,60.0,50.0,40.0,66.7,50.0,12.7,0.0,52.66,Lake Forest
"Arrish, Ezra, Joe, Leonardo, Thomas",3.85,7,6,1,1,1,1,3,1,2,4,3,6,1,2,0,2,3,1,3,0,0,0,2,3,50.00,50.00,50.00,0.00,7.0,5.2,1.00,1.15,10.39,4,2,50.0,75.0,66.7,25.0,50.0,33.3,36.1,0.0,73.01,Lake Forest
"Eamonn, Joe, Josh, Leonardo, Skyler",3.75,13,2,11,4,0,1,3,0,6,6,1,5,1,1,0,2,0,0,0,3,1,0,0,0,100.00,100.00,100.00,0.00,5.8,6.7,2.26,0.30,117.33,3,1,0.0,75.0,75.0,25.0,0.0,25.0,0.0,37.5,61.44,Lake Forest
"Ezra, Joe, Josh, Leonardo, Thomas",2.60,7,5,2,1,1,0,1,1,3,5,2,3,1,3,1,2,0,0,1,2,2,0,0,0,60.00,60.00,33.33,50.00,4.8,4.8,1.46,1.04,30.77,2,1,50.0,100.0,66.7,0.0,50.0,33.3,16.7,40.0,73.85,Lake Forest
"Elliot, Ezra, Joe, Josh, Leonardo",2.35,4,2,2,2,1,0,2,2,2,5,1,3,0,1,0,1,0,0,0,0,0,0,1,3,40.00,40.00,0.00,0.00,3.8,2.9,1.04,0.69,34.04,3,2,33.3,100.0,60.0,0.0,66.7,40.0,0.0,0.0,65.36,Lake Forest
"Joe, Josh, Leonardo, Skyler, Thomas",2.27,2,5,-3,0,1,0,3,1,1,3,2,5,0,0,1,2,0,0,3,0,0,1,0,0,33.33,33.33,0.00,50.00,4.8,4.8,0.42,1.04,-52.94,4,1,50.0,100.0,80.0,0.0,50.0,20.0,50.0,0.0,84.71,Lake Forest
"Arrish, Elliot, Joe, Leonardo, Thomas",2.03,5,7,-2,0,0,1,0,2,0,0,2,3,0,0,0,0,8,4,1,0,0,0,2,4,0.00,0.00,0.00,0.00,4.3,3.6,1.15,1.94,-39.34,0,3,0.0,0.0,0.0,100.0,100.0,100.0,22.1,0.0,85.36,Lake Forest
"Alec, Elliot, Ezra, Joe, Josh",1.87,3,2,1,1,0,0,2,2,1,3,1,3,1,3,0,2,0,0,0,0,0,0,0,0,33.33,33.33,33.33,0.00,2.9,2.9,1.04,0.69,21.43,2,2,0.0,100.0,50.0,0.0,100.0,50.0,0.0,0.0,61.71,Lake Forest
